In [1]:
opciones = ["piedra", "tijeras", "papel"]

In [40]:
def ganador(jugador1, jugador2):
    if jugador1 == jugador2:
        ganador = 0
    elif jugador1 == "piedra" and jugador2 == "tijeras":
        ganador = 1
    elif jugador1 == "piedra" and jugador2 == "papel":
        ganador = 2
    elif jugador1 == "tijeras" and jugador2 == "piedra":
        ganador = 2
    elif jugador1 == "tijeras" and jugador2 == "papel":
        ganador = 1
    elif jugador1 == "papel" and jugador2 == "piedra":
        ganador = 1
    elif jugador1 == "papel" and jugador2 == "tijeras":
        ganador = 2
    return ganador

In [3]:
from random import choice
def elegir_opcion():
    return choice(opciones)

In [41]:
for i in range(10):
    jugador1 = elegir_opcion()
    jugador2 = elegir_opcion()
    print("Jugador 1: %s Jugador 2: %s Ganador: %s " % (
        jugador1, jugador2,ganador(jugador1, jugador2)
    ))


Jugador 1: piedra Jugador 2: papel Ganador: 2 
Jugador 1: tijeras Jugador 2: piedra Ganador: 2 
Jugador 1: papel Jugador 2: papel Ganador: 0 
Jugador 1: papel Jugador 2: papel Ganador: 0 
Jugador 1: papel Jugador 2: piedra Ganador: 1 
Jugador 1: tijeras Jugador 2: piedra Ganador: 2 
Jugador 1: piedra Jugador 2: piedra Ganador: 0 
Jugador 1: piedra Jugador 2: tijeras Ganador: 1 
Jugador 1: tijeras Jugador 2: piedra Ganador: 2 
Jugador 1: piedra Jugador 2: papel Ganador: 2 


In [42]:
def binario(opcion):
    if opcion=="piedra":
        respuesta = [1,0,0]
    elif opcion=="tijeras":
        respuesta = [0,1,0]
    else:
        respuesta = [0,0,1]
    return respuesta

datos_1 = list(map(binario, ["piedra", "tijeras", "papel"]))
datos_2 = list(map(binario, ["papel", "piedra", "tijeras"]))

print(datos_1)
print(datos_2)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [15]:
from sklearn.neural_network import MLPClassifier

In [16]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [17]:
modelo = clf.fit([datos_1[0]], [datos_2[0]])
print(modelo)

MLPClassifier(warm_start=True)


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [43]:
def jugar_aprender(iters=10, debug=False):
    marcador = {"ganados": 0, "perdidos": 0}
    
    datos_1 = []
    datos_2 = []
    
    for i in range(iters):
        jugador1 = elegir_opcion()
        
        predecir = modelo.predict_proba([binario(jugador1)])[0]
        
        if predecir[0] >= 0.95:
            jugador2 = opciones[0]
        elif predecir[1] >= 0.95:
            jugador2 = opciones[1]
        elif predecir[2] >= 0.95:
            jugador2 = opciones[2]
        else:
            jugador2 = elegir_opcion()
            
        if debug==True:
            print("Jugador 1: %s jugador 2 (modelo): %s --> %s" % (jugador1, predecir, jugador2))
        
        ganador1 = ganador(jugador1, jugador2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % ganador1)
        
        if ganador1==2:
            datos_1.append(binario(jugador1))
            datos_2.append(binario(jugador2))
            
            marcador["ganados"]+=1
        else:
            marcador["perdidos"]+=1
        
    return marcador, datos_1, datos_2

In [51]:
marcador, datos_1, datos_2 = jugar_aprender(1, debug=True)
print(datos_1)
print(datos_2)
print("Marcador: %s %s %%" % (marcador, (marcador["ganados"]*100/(marcador["ganados"]+marcador["perdidos"]))))
if len(datos_1):
    modelo = modelo.partial_fit(datos_1, datos_2)

Jugador 1: piedra jugador 2 (modelo): [0.00444254 0.00456046 0.987797  ] --> papel
Comprobamos: p1 VS p2: 2
[[1, 0, 0]]
[[0, 0, 1]]
Marcador: {'ganados': 1, 'perdidos': 0} 100.0 %


In [52]:
i = 0
historial_pct = []
while True:
    i+=1
    marcador, datos_1, datos_2 = jugar_aprender(1000, debug=False)
    pct = (marcador["ganados"]*100/(marcador["ganados"]+marcador["perdidos"]))
    historial_pct.append(pct)
    print("Iteracion: %s - score: %s %s %%" % (i, marcador, pct))
    
    if len(datos_1):
        modelo= modelo.partial_fit(datos_1, datos_2)
    
    if sum(historial_pct[-9:])==900:
        break

Iteracion: 1 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 2 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 3 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 4 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 5 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 6 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 7 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 8 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %
Iteracion: 9 - score: {'ganados': 1000, 'perdidos': 0} 100.0 %


In [36]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [37]:
x = range(len(historial_pct))
y = historial_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iteracion", y_axis_label="%", width=1000)
p.line(x, y, line_width=1)

show(p)

In [55]:
print(modelo.predict_proba([binario("tijeras")]))
print(opciones)

[[0.97459116 0.02152259 0.0166361 ]]
['piedra', 'tijeras', 'papel']
